In [32]:
import unicodedata
import re
import pandas as pd
import torch
import torch.nn as nn
import random
import time
import math

In [33]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# 2. Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    # Put a space between a word and its punctuation
    s = re.sub(r"([.!?])", r" \1", s)
    # Remove anything that isn't a letter or punctuation
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [34]:
def load_data(filename):
    # Use the first two columns: English (source) and Spanish (target)
    df = pd.read_csv("spa.csv", sep='\t', header=None, usecols=[0, 1], names=['eng', 'spa'])
    
    # Apply normalization
    df['eng'] = df['eng'].apply(normalizeString)
    df['spa'] = df['spa'].apply(normalizeString)
    
    # IMPORTANT: We operate on PAIRS. 
    # The models need (Input, Target) tuples.
    pairs = df.values.tolist()
    
    return pairs

# Let's run it on our file
pairs = load_data('spa.csv')

print("--- Sample Pairs ---")
print(pairs[:5])

--- Sample Pairs ---
[['go .', 've .'], ['go .', 'vete .'], ['go .', 'vaya .'], ['go .', 'vayase .'], ['hi .', 'hola .']]


In [35]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [36]:
MAX_LENGTH = 10  # Easier for the Vanilla models to handle initially

def prepareData(lang1, lang2):
    pairs = load_data('spa.csv') # The function we wrote in the last step
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    
    print(f"Read {len(pairs)} sentence pairs")
    
    # Filter pairs (optional but recommended for speed)
    pairs = [p for p in pairs if len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH]
    
    print(f"Trimmed to {len(pairs)} sentence pairs")
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'spa')

Read 139013 sentence pairs
Trimmed to 106808 sentence pairs
Counted words:
eng 11372
spa 21824


In [37]:
class VanillaEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, rnn_type='GRU'):
        super(VanillaEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_type = rnn_type

        # 1. Embedding Layer
        self.embedding = nn.Embedding(input_size, hidden_size)

        # 2. RNN Layer (Switchable between GRU and LSTM)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(hidden_size, hidden_size)
        else:
            self.rnn = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # Reshape input to (1, 1, -1) because we are processing one word at a time 
        # (Batch size = 1 for this simple implementation)
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

    def initHidden(self, device):
        # Initialize hidden state with zeros
        # GRU needs 1 state, LSTM needs 2 (hidden + cell_state)
        if self.rnn_type == 'LSTM':
            return (torch.zeros(1, 1, self.hidden_size, device=device),
                    torch.zeros(1, 1, self.hidden_size, device=device))
        else:
            return torch.zeros(1, 1, self.hidden_size, device=device)

In [38]:
class VanillaDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, rnn_type='GRU'):
        super(VanillaDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_type = rnn_type

        self.embedding = nn.Embedding(output_size, hidden_size)
        
        # Switchable RNN
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(hidden_size, hidden_size)
        else:
            self.rnn = nn.GRU(hidden_size, hidden_size)
            
        # Output layer to map hidden state to vocabulary size
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = torch.relu(output) # Optional: add non-linearity
        
        output, hidden = self.rnn(output, hidden)
        
        # Project to vocabulary size and apply Softmax
        prediction = self.softmax(self.out(output[0]))
        return prediction, hidden

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Device found: {device}")

Device found: cuda


In [40]:
encoder = VanillaEncoder(input_lang.n_words, 256, rnn_type='GRU').to(device)
decoder = VanillaDecoder(256, output_lang.n_words, rnn_type='GRU').to(device)

In [41]:
encoder = VanillaEncoder(input_lang.n_words, 256, rnn_type='LSTM').to(device)
decoder = VanillaDecoder(256, output_lang.n_words, rnn_type='LSTM').to(device)

In [42]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# --- The Single Training Step ---
teacher_forcing_ratio = 0.5

In [43]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden(device)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(input_length, encoder.hidden_size, device=device)

    loss = 0

    # 1. Encode the input sentence
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        # encoder_outputs[ei] = encoder_output[0, 0] # Used for Attention later, not needed for Vanilla

    # 2. Decode the response
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden # Pass context vector

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [44]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = torch.optim.SGD(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss() 
    
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    print(f"Starting training on {device}...")
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(f"Iter: {iter} | Loss: {print_loss_avg:.4f}")

In [45]:
print("Training Model 1: Vanilla GRU")
hidden_size = 256
encoder1 = VanillaEncoder(input_lang.n_words, hidden_size, rnn_type='GRU').to(device)
decoder1 = VanillaDecoder(hidden_size, output_lang.n_words, rnn_type='GRU').to(device)

trainIters(encoder1, decoder1, 20000, print_every=1000)

Training Model 1: Vanilla GRU
Starting training on cuda...
Iter: 1000 | Loss: 5.2171
Iter: 2000 | Loss: 4.9971
Iter: 3000 | Loss: 4.9681
Iter: 4000 | Loss: 4.8705
Iter: 5000 | Loss: 4.7239
Iter: 6000 | Loss: 4.6670
Iter: 7000 | Loss: 4.5493
Iter: 8000 | Loss: 4.4662
Iter: 9000 | Loss: 4.4978
Iter: 10000 | Loss: 4.4310
Iter: 11000 | Loss: 4.2778
Iter: 12000 | Loss: 4.2454
Iter: 13000 | Loss: 4.2080
Iter: 14000 | Loss: 4.1095
Iter: 15000 | Loss: 4.1245
Iter: 16000 | Loss: 4.0283
Iter: 17000 | Loss: 3.9648
Iter: 18000 | Loss: 4.0224
Iter: 19000 | Loss: 3.9307
Iter: 20000 | Loss: 3.8596


In [46]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden(device)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.detach()

        return decoded_words

In [47]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# Check the results
evaluateRandomly(encoder1, decoder1)

> how long have you been here in boston ?
=  cuanto tiempo llevas aqui en boston ?
<  cuanto cuanto en una vez ? <EOS>

> the town where i was born is beautiful .
= la ciudad en que naci es muy bonita .
< el se se esta el de . <EOS>

> do you want to go for a swim ?
=  quieren ir a nadar ?
<  quieres un una de una ? <EOS>

> my eyes aren t blue .
= mis ojos no son azules .
< mi no no es . . <EOS>

> i need a favor .
= necesito un favor .
< necesito un una . <EOS>

> we love tom .
= amamos a tom .
< tomas a tom . <EOS>

> do you know whose bike this is ?
=  saben de quien es esta bici ?
<  que esta es esta ? ? <EOS>

> what do you have on the menu today ?
=  que tienes en el menu hoy ?
<  que te que hoy hoy ? <EOS>

> i prefer to go barefoot in the house .
= prefiero andar descalzo por la casa .
< yo a a en casa a casa . <EOS>

> i ll leave tomorrow weather permitting .
= me ire manana si el tiempo lo permite .
< me a de las . . <EOS>



In [48]:
print("Training Model 2: Vanilla LSTM")

# Re-initializing with LSTM flag
# We keep hidden_size at 256 for a fair comparison
encoder2 = VanillaEncoder(input_lang.n_words, 256, rnn_type='LSTM').to(device)
decoder2 = VanillaDecoder(256, output_lang.n_words, rnn_type='LSTM').to(device)

# Using the same training loop
trainIters(encoder2, decoder2, 20000, print_every=1000)


Training Model 2: Vanilla LSTM
Starting training on cuda...
Iter: 1000 | Loss: 5.3450
Iter: 2000 | Loss: 5.0553
Iter: 3000 | Loss: 5.0055
Iter: 4000 | Loss: 4.9715
Iter: 5000 | Loss: 4.8759
Iter: 6000 | Loss: 4.8065
Iter: 7000 | Loss: 4.7674
Iter: 8000 | Loss: 4.6766
Iter: 9000 | Loss: 4.6526
Iter: 10000 | Loss: 4.5506
Iter: 11000 | Loss: 4.5253
Iter: 12000 | Loss: 4.4939
Iter: 13000 | Loss: 4.3744
Iter: 14000 | Loss: 4.3185
Iter: 15000 | Loss: 4.3535
Iter: 16000 | Loss: 4.2704
Iter: 17000 | Loss: 4.2435
Iter: 18000 | Loss: 4.1633
Iter: 19000 | Loss: 4.1796
Iter: 20000 | Loss: 4.0792


In [49]:
# Evaluate immediately after
print("\n--- Evaluation: Vanilla LSTM ---")
evaluateRandomly(encoder2, decoder2)


--- Evaluation: Vanilla LSTM ---
> i m undressing .
= me estoy desnudando .
< estoy . . . <EOS>

> i want something cold to drink now .
= ahora quiero tomar algo fresco .
< quiero quiero que algo . <EOS>

> tom doesn t know where to look .
= tom no sabe a donde mirar .
< tom no sabe a tom . <EOS>

> this isn t my wallet .
= esta no es mi cartera .
< esta no es mi . . <EOS>

> she believes that he is innocent .
= ella piensa que el es inocente .
< ella es lo que . . <EOS>

> how is everything ?
=  como va todo ?
<  como es el ? ? ? ? ? ?

> tom walked over to where mary was standing .
= tom camino hacia donde mary estaba parada .
< tom tom que mary mary mary . . . <EOS>

> is there a good restaurant somewhere near here ?
=  hay un buen restaurante por aqui cerca ?
< hay hay aqui ? <EOS>

> all the hostages were released unharmed .
= todos los rehenes fueron liberados ilesos .
< la los se en el . . . <EOS>

> it seems obvious that he is sick .
= parece obvio que el esta enfermo .
< se l

In [50]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(encoder, decoder, n_sentences=100):
    print(f"Calculating BLEU score on {n_sentences} random sentences...")
    references = []
    candidates = []

    for i in range(n_sentences):
        pair = random.choice(pairs)
        input_sentence = pair[0]
        target_sentence = pair[1]
        
        # Get the model's prediction
        output_words = evaluate(encoder, decoder, input_sentence)
        
        # Remove <EOS> for cleaner comparison
        if '<EOS>' in output_words:
            output_words.remove('<EOS>')
            
        # NLTK expects a list of words for the candidate
        candidates.append(output_words)
        
        # NLTK expects a list of lists of words for references (in case there are multiple correct translations)
        # We only have one reference per input
        ref_words = target_sentence.split(' ')
        references.append([ref_words])

    # Calculate BLEU-1 to BLEU-4
    score = corpus_bleu(references, candidates)
    print(f"BLEU Score: {score:.4f}")
    return score

In [57]:
print("--- Model 1: Vanilla GRU ---")
bleu_gru = calculate_bleu(encoder1, decoder1)

print("\n--- Model 2: Vanilla LSTM ---")
bleu_lstm = calculate_bleu(encoder2, decoder2)

--- Model 1: Vanilla GRU ---
Calculating BLEU score on 100 random sentences...
BLEU Score: 0.0368

--- Model 2: Vanilla LSTM ---
Calculating BLEU score on 100 random sentences...
BLEU Score: 0.0303


In [52]:
import nltk
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_metric

In [53]:
nltk.download('wordnet')
nltk.download('omw-1.4')

def calculate_advanced_metrics(encoder, decoder, n_sentences=100):
    print(f"Calculating METEOR & BERTScore on {n_sentences} samples...")
    references = []  # For METEOR: list of lists of tokens
    candidates = []  # For METEOR: list of tokens
    
    # For BERTScore we need lists of full strings
    ref_strings = []
    cand_strings = []

    for i in range(n_sentences):
        pair = random.choice(pairs)
        input_sentence = pair[0]
        target_sentence = pair[1]
        
        # Get Prediction
        output_words = evaluate(encoder, decoder, input_sentence)
        if '<EOS>' in output_words: output_words.remove('<EOS>')
        pred_str = " ".join(output_words)
        
        # Prepare data structures
        # METEOR expects: reference=[['this', 'is', 'ref']], candidate=['this', 'is', 'pred']
        references.append([target_sentence.split()]) 
        candidates.append(output_words)
        
        # BERTScore expects: references=['string'], candidates=['string']
        ref_strings.append(target_sentence)
        cand_strings.append(pred_str)

    # 1. Calculate METEOR (Average over corpus)
    meteor_total = 0
    for ref, cand in zip(references, candidates):
        meteor_total += meteor_score(ref, cand)
    avg_meteor = meteor_total / len(references)
    print(f"METEOR Score: {avg_meteor:.4f}")

    # 2. Calculate BERTScore (Returns Precision, Recall, F1)
    # lang='es' because our target is Spanish
    P, R, F1 = bert_metric(cand_strings, ref_strings, lang='es', verbose=False)
    print(f"BERTScore F1: {F1.mean().item():.4f}")

[nltk_data] Downloading package wordnet to /home/goktug/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/goktug/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [54]:
print("--- Advanced Metrics: Vanilla GRU ---")
calculate_advanced_metrics(encoder1, decoder1)

print("\n--- Advanced Metrics: Vanilla LSTM ---")
calculate_advanced_metrics(encoder2, decoder2)

--- Advanced Metrics: Vanilla GRU ---
Calculating METEOR & BERTScore on 100 samples...
METEOR Score: 0.2492
BERTScore F1: 0.7207

--- Advanced Metrics: Vanilla LSTM ---
Calculating METEOR & BERTScore on 100 samples...
METEOR Score: 0.2193
BERTScore F1: 0.6942
